In [12]:
# makes data to try LR on for completeness/purity

In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from astropy.io import fits

import random

import sklearn
from sklearn.linear_model import LinearRegression

%matplotlib inline

# Function definitions

In [2]:
f = '/Users/josephwick/Documents/Data/merian/COSMOS2015_r23.6_SOM_v3.fits'
wtsf = '../speczWeights/specz-weights.npy'

## Getting weighted specz set

In [3]:
def getSpeczData(fname):
    data = fits.open(fname)[1].data
    data = data[data['z_type']==1]

    aphot = 10**(-0.4 * np.c_[data['a_g'], data['a_r'], data['a_i'], 
                                data['a_z'], data['a_y']])

    spec_data = np.c_[data['g_cmodel_mag'],
                     data['r_cmodel_mag'],
                     data['i_cmodel_mag'],
                     data['z_cmodel_mag'],
                     data['y_cmodel_mag']] / aphot

    err = np.c_[data['g_cmodel_flux_err'],
                      data['r_cmodel_flux_err'],
                      data['i_cmodel_flux_err'],
                      data['z_cmodel_flux_err'],
                      data['y_cmodel_flux_err']] / aphot

    mask_calib = (err > 0.) & np.isfinite(err) & np.isfinite(spec_data)

    # remove any sources with non-detections
    sel = mask_calib.sum(axis=1) == 5
    spec_data = spec_data[sel]
    
    # weight 
    wts = np.load(wtsf)
    spec_data_wtd = weightData(spec_data, wts)
    
    # masses
    masses = data['mass_cosmos']
    masses = masses[sel]
    
    return spec_data_wtd, masses

In [4]:
def weightData(data, wts):
    mag_data_wtd = []
    for i in range(5):
        mag_data_wtd.append(data[:,1] *wts )
    
    return mag_data_wtd

In [5]:
# merian range:
#   z between 0.058 and 0.1
#   log mass between 8 and 9 solar masses

## Getting Merian cut

In [6]:
def getMerianRaw(fname):

    data = fits.open(fname)[1].data

    aphot = 10**(-0.4 * np.c_[data['a_g'], data['a_r'], data['a_i'], 
                                data['a_z'], data['a_y']])

    broad_data = np.c_[data['g_cmodel_mag'],
                     data['r_cmodel_mag'],
                     data['i_cmodel_mag'],
                     data['z_cmodel_mag'],
                     data['y_cmodel_mag']] / aphot

    err = np.c_[data['g_cmodel_flux_err'],
                      data['r_cmodel_flux_err'],
                      data['i_cmodel_flux_err'],
                      data['z_cmodel_flux_err'],
                      data['y_cmodel_flux_err']] / aphot

    mask_calib = (err > 0.) & np.isfinite(err) & np.isfinite(broad_data)

    # remove any sources with non-detections
    sel = mask_calib.sum(axis=1) == 5
    broad_data = broad_data[sel]

    zees = data['z']
    zees = zees[sel]

    masses = data['mass_cosmos']
    masses = masses[sel]
    
    #reshape 
    merian_data = []
    for i in range(5):
        merian_data.append( broad_data[:,1] )
        
    # zees and mass
    sel = np.where( 0.058 <= zees)
    zees = zees[sel]
    for i in range(len(merian_data)):
        merian_data[i] = merian_data[i][sel[0]]
    masses=masses[sel]
        
    sel = np.where( zees <= 0.1)
    zees = zees[sel]
    for i in range(len(merian_data)):
        merian_data[i] = merian_data[i][sel[0]]
    masses = masses[sel]
    
    # make mass/z cut
    sel = np.where(8 <= masses)
    masses = masses[sel]
    for i in range(len(merian_data)):
        merian_data[i] = merian_data[i][sel]
    
    sel = np.where(masses <= 9)
    masses = masses[sel]
    for i in range(len(merian_data)):
        merian_data[i] = merian_data[i][sel]
        
    return merian_data, masses

In [7]:
# first, completeness
# remove some things according to their mass

In [57]:
def isDetected(m) :
    #p = 0.2 + 0.8*(m-8) #linear with mass
    #p = 1 - (m-8.5)**2 #parabolic with mass
    p = random.uniform(0,1) #random
    if m>=7.9: p=1.0
    
    r = random.uniform(0,1)
    
    if r>p:
        return False
    else:
        return True

In [9]:
def makeCompletenessCut(data, ms):
    remove = []
    data_c = []
    for i in range(len(data[1])):
        isDet = isDetected(ms[i])
        if not isDet:
            remove.append(i)

    for i in range(len(data)):
        data_c.append(np.delete(data[i], remove))
    masses = np.delete(ms, remove)

    return data_c, masses

In [10]:
# now, purity
# add in some random galaxies from z~1

In [58]:
def addImpurity(m):
    # p = 0.6 +0.3*(m-8) #linear with mass
    p = random.uniform(0,1) #random
    
    r = random.uniform(0,1)
    
    if r>p:
        return True
    else:
        return False

In [12]:
def makePurityCut(dataC, fname, masses):

    data = fits.open(fname)[1].data

    aphot = 10**(-0.4 * np.c_[data['a_g'], data['a_r'], data['a_i'], 
                                data['a_z'], data['a_y']])

    broad_data = np.c_[data['g_cmodel_mag'],
                     data['r_cmodel_mag'],
                     data['i_cmodel_mag'],
                     data['z_cmodel_mag'],
                     data['y_cmodel_mag']] / aphot

    err = np.c_[data['g_cmodel_flux_err'],
                      data['r_cmodel_flux_err'],
                      data['i_cmodel_flux_err'],
                      data['z_cmodel_flux_err'],
                      data['y_cmodel_flux_err']] / aphot

    mask_calib = (err > 0.) & np.isfinite(err) & np.isfinite(broad_data)
    
    zed = np.array(data['z'])
    
    # remove any sources with non-detections
    sel = mask_calib.sum(axis=1) == 5
    broad_data = broad_data[sel]
    zed = zed[sel]
    
    # get galaxies at z = 1ish
    data = broad_data[zed > 0.8]
    zed = zed[zed>0.8]
    additives = data[zed < 1.2]
    data_p=[]
    gees=[]
    mazzes = np.copy(masses)
    added_masses=[]
    
    for i in range(len(dataC[1])):
        addImp = addImpurity(masses[i])
        if addImp:
            j = random.randint(0, len(additives)-1)
            g = np.c_[additives[:,0][j], additives[:,1][j], additives[:,2][j], additives[:,3][j], 
                      additives[:,4][j]]
            gees.append(g[0])
            added_masses.append(masses[i])
            mazzes = np.append(mazzes, masses[i])
       
    gees = np.array(gees)
    for k in range(len(dataC)):
        data_p.append(np.append(dataC[k], gees[:,k]))
    
    return data_p, added_masses, mazzes

## Get LR style data

In [13]:
def getLRData():

    # representative 
    data_specz, massesSpec = getSpeczData(f)
    # merian
    rawMerian, rawMasses = getMerianRaw(f)
    merianC, massesC = makeCompletenessCut(rawMerian, rawMasses)
    data_merian, added_masses, massesCP = makePurityCut(merianC, f, massesC)
    #np-ify 
    data_merian = np.array(data_merian)
    data_specz = np.array(data_specz)

    # get ground truths
    # first we need to bin by mass
    nbins = 10
    slices = np.linspace(rawMasses.min(), rawMasses.max(), nbins, True)

    cIdxs = np.digitize(massesC, slices)-1
    rawIdxs = np.digitize(rawMasses, slices)-1
    addedIdxs = np.digitize(added_masses, slices)-1
    cpIdxs = np.digitize(massesCP, slices)-1
    
    specIdxs = np.digitize(massesSpec, slices)-1

    # completeness is len(merianC)/len(rawMerian)
    # purity is len(rawMerian)/len(rawMerian+added)
    comps= []
    purs = []
    for i in range(nbins):
        b_c = np.array(data_merian[0])[np.where(cIdxs == i)]
        b_raw = np.array(rawMerian[0])[np.where(rawIdxs == i)]
        b_am = np.array(added_masses)[np.where(addedIdxs == i)]
        b_cp = np.array(massesCP)[np.where(cpIdxs == i)]
    
        comps.append(len(b_c)/len(b_raw))
        if len(b_cp >0):
            purs.append(1-(len(b_am)/len(b_cp)))
        else:
            purs.append(0)
    
    data = []
    labels = purs
    for b in range(nbins):
        fig, axs = plt.subplots(1,5, figsize=(25,5))

        heights_m = []
        heights_s = []
        for band in range(len(data_merian)):
            mer = data_merian[band][np.where(cpIdxs == b)]
            spec= data_specz[band][np.where(specIdxs == b)]
    
            h, bins, _ = axs[band].hist(mer, density = True, bins=8)
            heights_m.append(h)
    
            h, _, _ = axs[band].hist(spec, density=True, bins=bins)
            heights_s.append(h)
    
            plt.close()
            heights_s[band], heights_m[band]
            
        binData_s = np.array(heights_s).flatten()
        binData_m = np.array(heights_m).flatten()
    
        div = binData_s/binData_m
        div[div>7] = 7
        data.append(np.nan_to_num(div))
        
    return data, labels

In [14]:
def getLRDatas(n):
    nData = []
    nLabels = []
    
    for i in range(n):
        iData, iLabels = getLRData()
        for d in iData:
            nData.append(d)
        for l in iLabels:
            nLabels.append(l)
    
    return np.array(nData), np.array(nLabels)

# Make Data

In [65]:
data, labels = getLRDatas(1)

/var/folders/js/qxrb9plj563_v0syz958yd8h0000gn/T/ipykernel_13704/4039878333.py:64: RuntimeWarning: invalid value encountered in true_divide
  div = binData_s/binData_m
/var/folders/js/qxrb9plj563_v0syz958yd8h0000gn/T/ipykernel_13704/4039878333.py:64: RuntimeWarning: divide by zero encountered in true_divide
  div = binData_s/binData_m


In [66]:
dataSave = 'data/smDiv_n10_data-p.npy'
lblSave = 'data/smDiv_n10_label-p.npy'

In [67]:
np.save(dataSave, data)
np.save(lblSave, labels)